# AMADEUS CHALLENGE 

## Archivos de trabajo: bookings.csv y searches.csv


### -Ejercicio 1: Count the number of lines in Python for each file

### Resuelto con la Shell

In [ ]:
#sacamos un sample para trabajar sobre él con mayor comodidad y agilidad

In [1]:
! bzcat ./bookings.csv.bz2 | head -10000 > ./bookings.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./bookings.csv.bz2, output file = (stdout)


- bzip2 otro compresor para archivos grandes, garantiza que dentro solo hay un archivo por lo que puedes trabajar sobre el comprimido sabiendo qué archivo estás modificando

In [ ]:
# ! bzip2 nombre_fichero para comprimir, ! bunzip2 *.bz2 para descomprimir

In [2]:
! bzip2 -f ./bookings.sample.csv #-fpara que no de error al ejecutar varias veces

In [ ]:
# hemos pasado el sample otra vez al formato original por costumbre, no es obligatorio

In [ ]:
pwd

In [3]:
! bzcat ./bookings.sample.csv.bz2 | wc -l

10000


### Resuelto con Python

In [ ]:
- a)sin descomprimir
- b)using row uncompressed files

In [4]:
import bz2

In [5]:
fileBz2 = bz2.BZ2File('./bookings.sample.csv.bz2')

In [6]:
type(fileBz2)

bz2.BZ2File

In [7]:
k = 0
for line in fileBz2:
    k+=1
print(k) 

10000


-Buena practica unir en una sola celda lo que comprobamos que funciona hasta ahora

In [8]:
fileBz2 = bz2.BZ2File('./bookings.sample.csv.bz2')
k = 0
for line in fileBz2:
    k+=1
print(k) 

10000


-importante cerrar fichero porque si estamos leyendo muchos nos consume la memoria

In [9]:
fileBz2.close()

In [10]:
fileBz2.closed

True

-Mejor usar with para que se cierre automaticamente el archivo y no nos preocupamos

In [11]:
with bz2.BZ2File('./bookings.sample.csv.bz2') as fileBz2:
    k = 0
    for line in fileBz2:
        k+=1
    print(k) 

10000


In [12]:
fileBz2.closed

True

-Leemos y cerramos el fichero con try y except para evitar que nos de error, y cubrir el caso de que el archivo no exista, como en el siguiente ejemplo:

In [13]:
try:
    with bz2.BZ2File('./bookingsj.sample.csv.bz2') as fileBz2:
        k = 0
        for line in fileBz2:
            k+=1
        print(k) 
except FileNotFoundError:
    print('File not found')
except:
    print('unexpected error')

File not found


-Esta es la forma más profesional de hacerlo

In [14]:
try:
    with bz2.BZ2File('./bookingsj.sample.csv.bz2') as fileBz2:
        k = 0
        for line in fileBz2:
            k+=1
        print(k) 
except FileNotFoundError as message:
    print(message)
except:
    print('unexpected error')

[Errno 2] No such file or directory: './bookingsj.sample.csv.bz2'


- Codigo aún mas profesional

In [15]:
try:
    with bz2.BZ2File('./bookings.sample.csv.bz2') as fileBz2:
        k = 0
        for k,line in enumerate(fileBz2):
                pass #no hace nada
        print(k+1) 
except FileNotFoundError as message:
    print(message)
except:
    print('unexpected error')

10000


### Resuelto con Pandas

In [16]:
import pandas as pd

-Sacamos los nombres de las columnas

In [18]:
! bzcat bookings.sample.csv.bz2 | head -1 | tr ^ '\n'

act_date           
source
pos_ctry
pos_iata
pos_oid  
rloc          
cre_date           
duration
distance
dep_port
dep_city
dep_ctry
arr_port
arr_city
arr_ctry
lst_port
lst_city
lst_ctry
brd_port
brd_city
brd_ctry
off_port
off_city
off_ctry
mkt_port
mkt_city
mkt_ctry
intl
route          
carrier
bkg_class
cab_class
brd_time           
off_time           
pax
year
month
oid      

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = bookings.sample.csv.bz2, output file = (stdout)


-Elegimos solo una columna para no leer el fichero entero

In [19]:
df = pd.read_csv('bookings.sample.csv.bz2', sep='^', usecols = ['pax'])

-Nos saldrán 9999 líneas porque una es la cabecera

In [20]:
df.shape

(9999, 1)

In [21]:
len(df)

9999

### -Ejercicio 2: Plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona

-inicialmente yo lei el archivo con el atributo nrrows 10.000 lineas sobre el fichero original en lugar de sobre el sample, pero también se puede hacer trabajando directamente con el sample, es lo mismo

In [22]:
import pandas as pd

In [23]:
b = pd.read_csv('bookings.sample.csv.bz2', sep='^') 

#        PLAN DE ACCION:

## 1. Nos familiarizamos con los datos

In [24]:
b.shape

(9999, 38)

In [25]:
b.describe() # nos devuelve solo las columnas que son numéricas

,duration,distance,intl,pax,year,month
count,9999.000000,9999.000000,9999.000000,9999.000000,9999.0,9999.0
mean,20209.746075,455.608861,0.596260,0.516152,2013.0,3.0
std,42243.233482,1738.891068,0.490671,1.783970,0.0,0.0
min,59.000000,0.000000,0.000000,-25.000000,2013.0,3.0
25%,3339.000000,0.000000,0.000000,-1.000000,2013.0,3.0
50%,7721.000000,0.000000,1.000000,1.000000,2013.0,3.0
75%,19077.000000,0.000000,1.000000,1.000000,2013.0,3.0
max,488131.000000,18481.000000,1.000000,25.000000,2013.0,3.0


In [26]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


-La función sample te devuelve 5 filas aleatorias del archivo

In [27]:
b.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
9079,2013-03-06 00:00:00,1P,LB,fb1e34ecaf06a71d3b38e9da8543f310,24856a47570f69253fd12224fa198cc0,8766ee850e14f10e2311461061462d58,2013-03-05 00:00:00,15082,0,BEY,...,CDGBEY,KP,T,Y,2013-03-25 16:20:00,2013-03-25 21:52:49,-1,2013,3,NULL
9131,2013-03-20 00:00:00,1P,SE,4425c32249b69d5e2cb9a4c2db2a761e,d4e2f2dd948cf1fefe04d9e41523fe1c,f54a261f51ccd2216b2bf6d5dfde4003,2013-03-20 00:00:00,44522,0,CPH,...,CPHDXBPEW,OU,K,Y,2013-04-01 14:55:00,2013-04-02 08:47:28,1,2013,3,NULL
866,2013-03-13 00:00:00,1A,CH,a3ebf7f83905f86ff2455bfaff5226ef,79102c6e83d21d6633c7554b817c07a0,bd69f0ce6e59a4b14d39d6e8042e7fab,2013-03-13 00:00:00,1743,0,DUS,...,FCODUS,VR,Z,C,2013-04-24 14:10:00,2013-04-24 16:18:05,1,2013,3,NULL
7259,2013-03-14 00:00:00,1A,CA,ca08194c774ff0ec180ea682e2b9887f,ff21a3fda04829d23da48c1d98564c52,f00a6c49f77d86b608ccfe997cb24b81,2013-03-12 00:00:00,57304,0,YVR,...,TPEYVR,LC,U,Y,2013-05-01 23:55:00,2013-05-01 20:59:38,-1,2013,3,YVR6W211N
6562,2013-03-15 00:00:00,1A,DE,329c96758f09971c75db31792c1c2b94,6ab00ec6f3910d2b6cac4644937e2187,5ccc35fe371c0c0beb4e44f0bed3eed5,2013-03-11 00:00:00,7103,0,FRA,...,BEYFRA,WO,Q,Y,2013-03-25 07:15:00,2013-03-25 10:23:17,1,2013,3,FRAER2211


-Estos comandos (head,sample,tail) no nos sirve porque no podemos ver las 38 columnas, para eso tenemos el set_option, con None no hay limite de columnas, si ponemos numero esas son las que nos enseñan

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
b.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
938,2013-03-13 00:00:00,1A,FR,9451417efc0f2fbb5e49ecd42ebcd1fc,8d4f2bcba2a3fb8975c1a6f43b302cba,37d6f66916f7a334def0ba9eda39c13d,2013-03-12 00:00:00,11896,8759,BCN,BCN,ES,GRU,SAO,BR,GRU,SAO,BR,LIN,MIL,IT,GRU,SAO,BR,GRULIN,MILSAO,BRIT,1,LINCDGGRU,KP,M,Y,2013-05-17 06:55:00,2013-05-17 17:21:26,-4,2013,3,NULL
6288,2013-03-04 00:00:00,1V,US,befbdd06f3094f0b0c22c12a4b08eaa3,03afca6275a4c9450a2c34d1a23492cd,52b0942e534b970ed7729927b7330b50,2013-03-04 00:00:00,10581,0,PHL,PHL,US,MBJ,MBJ,JM,PHL,PHL,US,PHL,PHL,US,MBJ,MBJ,JM,MBJPHL,MBJPHL,JMUS,1,PHLMBJ,FD,S,Y,2013-08-04 07:55:00,2013-08-04 10:31:35,1,2013,3,NULL
2929,2013-03-11 00:00:00,1V,US,702d65dd8744f8ec09bda3e611f00a64,64a48fec9bafa3841b04caa9e9a2ca3d,be037395369ffd9fe1cda5175dcd2f86,2013-03-11 00:00:00,14759,0,EWR,NYC,US,PVG,SHA,CN,EWR,NYC,US,PVG,SHA,CN,EWR,NYC,US,EWRPVG,NYCSHA,CNUS,1,PVGEWR,FK,U,Y,2013-03-28 16:20:00,2013-03-28 19:04:38,1,2013,3,NULL
4892,2013-03-07 00:00:00,1P,US,76dd1c8493f2b4895443386566b7f317,db53267cd9741b4ecf03045460be7d1d,e536d48bb09dc820f0199b92b02ae883,2013-03-05 00:00:00,6046,0,MSY,MSY,US,CHS,CHS,US,MSY,MSY,US,CHS,CHS,US,MSY,MSY,US,CHSMSY,CHSMSY,USUS,0,CHSATLMSY,NV,T,Y,2013-03-23 17:00:00,2013-03-23 20:01:16,-1,2013,3,NULL
3102,2013-03-23 00:00:00,1P,US,82a19f9ce4e958cc4418b38c283ba844,bdc302c8855b41d81976b174c049516b,b68b506085bc06f0416d056d78afb7b2,2013-01-02 00:00:00,9425,0,DFW,DFW,US,PUJ,PUJ,DO,DFW,DFW,US,PUJ,PUJ,DO,DFW,DFW,US,DFWPUJ,DFWPUJ,DOUS,1,PUJATLDFW,NV,K,Y,2013-05-08 14:10:00,2013-05-08 20:05:53,3,2013,3,NULL


-Primero analizamos los datos. Por ejemplo tenemos 4 columnas con datetime. Act_date es la fecha de la ultima actividad, cre_date es cuando se crea la reserva.
La source es el operador que lo gestiona, el GDS (como Amadeus). Pos_iata son cosas hasheadas, que han pasado por un algoritmo y devuelve esos numeros, no se puede volver a un paso previo y ver de donde viene, las compañias lo utilizan para protegerse. Distance =0 representan los ida y vuelta, salen y vuelven al mismo sitio. Columna route, iniciales de la ruta realizada, cada ciudad son 3 letras, si hay 9 significa que ha habido una conexion

In [30]:
b.groupby('rloc')['act_date'].count().sort_values(ascending=False)

KeyError: 'rloc'

In [31]:
list(b.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

-el nombre de la columna tenia espacios por eso no lo reconoce

-rloc es el numero de reserva

In [32]:
b.groupby('rloc          ')['act_date           '].count().sort_values(ascending=False)

rloc          
ae15bcfc5aec0eb64b2c5204d08201d5    42
fb72a3899ed1cd353c5830388935e7f5    23
cd96f7b7fdb5743769053ba273c7eb5f    20
6b7878dd4ac59772e14ab4760ab45ad0    20
68aee71ee0f40aff44ed341f2c5f8627    16
2a86eac3e29c922f4a439fbc0480985b    15
58ee9c6852513816a39363a1621a0615    14
503d8dde8034a48c262b3f5764fc60ca    14
c9f19404e4f0755c40deaebd6e90ea84    13
182485c12b7e38aa6e2d24f7484c019b    12
cdf795b1e5710dc813c8b1147b427827    12
f3a5185d14eaa5258320bac03c5d9fc0    12
02ab3e3ded19a8cc6eb67c4413debb86    12
b8410227afd9eff71f9ed9c0e7013ddf    12
045e1c73107a2e4a39013d60e9b45aa3    12
a37584d1485cb35991e4ff1a2ba92262    12
f25cef4ed37d1483d0c4c7cfba9758ef    12
b0cd490450b69694ccafe0e08dfd821f    11
bab3d4e3fb3bee5e88340e7c359c6f13    11
2da897a9523b22f3e2cb9cd7099e0639    11
399664d8282d0587ce97c540d17eb06c    11
b0f878b0bbe51626016694032523295b    10
1e891061725828651e3e2943b79c6585    10
49edae7a10b0a8e2b2529ecc45e5dc74    10
45b87519d2fe4e85fd90512876fe9160    10
a27c351c88

In [33]:
b[b['rloc          '] == 'fb72a3899ed1cd353c5830388935e7f5'].sort_values('act_date           ',ascending=True).head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
5067,2013-03-01 00:00:00,1V,US,de1198ffef9fa56ba9fe841bac32bce8,60a531d07998240e3a5cbc207c5825e6,fb72a3899ed1cd353c5830388935e7f5,2013-02-25 00:00:00,14364,0,PHX,PHX,US,HKT,HKT,TH,PHX,PHX,US,PHX,PHX,US,HKT,HKT,TH,HKTPHX,HKTPHX,THUS,1,PHXLAXICNHKT,YK,K,Y,2013-08-30 20:00:00,2013-09-01 12:07:46,-2,2013,3,NULL
5068,2013-03-01 00:00:00,1V,US,de1198ffef9fa56ba9fe841bac32bce8,60a531d07998240e3a5cbc207c5825e6,fb72a3899ed1cd353c5830388935e7f5,2013-02-25 00:00:00,14364,0,PHX,PHX,US,HKT,HKT,TH,PHX,PHX,US,HKT,HKT,TH,CNX,CNX,TH,CNXHKT,CNXHKT,THTH,0,HKTCNX,EQ,Y,Y,2013-09-03 13:35:00,2013-09-03 15:46:42,-2,2013,3,NULL
5069,2013-03-01 00:00:00,1V,US,de1198ffef9fa56ba9fe841bac32bce8,60a531d07998240e3a5cbc207c5825e6,fb72a3899ed1cd353c5830388935e7f5,2013-02-25 00:00:00,14364,0,PHX,PHX,US,HKT,HKT,TH,PHX,PHX,US,CNX,CNX,TH,BKK,BKK,TH,BKKCNX,BKKCNX,THTH,0,CNXBKK,EQ,Y,Y,2013-09-05 15:05:00,2013-09-05 16:35:01,-2,2013,3,NULL
5070,2013-03-01 00:00:00,1V,US,de1198ffef9fa56ba9fe841bac32bce8,60a531d07998240e3a5cbc207c5825e6,fb72a3899ed1cd353c5830388935e7f5,2013-02-25 00:00:00,14364,0,PHX,PHX,US,HKT,HKT,TH,PHX,PHX,US,BKK,BKK,TH,ICN,SEL,KR,BKKICN,BKKSEL,KRTH,1,BKKICN,YK,E,Y,2013-09-08 23:40:00,2013-09-09 06:46:43,-2,2013,3,NULL
5071,2013-03-01 00:00:00,1V,US,de1198ffef9fa56ba9fe841bac32bce8,60a531d07998240e3a5cbc207c5825e6,fb72a3899ed1cd353c5830388935e7f5,2013-02-25 00:00:00,14364,0,PHX,PHX,US,HKT,HKT,TH,PHX,PHX,US,ICN,SEL,KR,PHX,PHX,US,ICNPHX,PHXSEL,KRUS,1,ICNLAXPHX,YK,E,Y,2013-09-09 20:20:00,2013-09-09 19:24:55,-2,2013,3,NULL


In [34]:
b.describe(include='all')

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
count,9999,9999,9999,9999,9999,9999,9999,9999.000000,9999.000000,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999.000000,9999,9999,9999,9999,9999,9999,9999.000000,9999.0,9999.0,9999
unique,31,5,112,2089,2271,3638,140,NaN,NaN,581,554,127,663,634,152,581,550,126,770,732,162,781,744,162,3395,3223,926,NaN,6079,179,27,4,6416,7218,NaN,NaN,NaN,389
top,2013-03-12 00:00:00,1A,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,ae15bcfc5aec0eb64b2c5204d08201d5,2013-03-12 00:00:00,NaN,NaN,LAX,NYC,US,LHR,NYC,US,LAX,NYC,US,LAX,NYC,US,LAX,NYC,US,JEDKHI,JEDKHI,USUS,NaN,KHIJED,NV,Q,Y,2013-06-09 15:10:00,2013-06-10 12:55:31,NaN,NaN,NaN,NULL
freq,768,3200,4839,435,435,42,1018,NaN,NaN,273,507,4600,212,391,3738,281,502,4559,228,435,4118,247,460,4170,52,52,3074,NaN,26,1354,851,8979,21,21,NaN,NaN,NaN,8427
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20209.746075,455.608861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.596260,NaN,NaN,NaN,NaN,NaN,NaN,0.516152,2013.0,3.0,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42243.233482,1738.891068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.490671,NaN,NaN,NaN,NaN,NaN,NaN,1.783970,0.0,0.0,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-25.000000,2013.0,3.0,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3339.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,2013.0,3.0,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7721.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2013.0,3.0,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19077.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2013.0,3.0,NaN


-Con este describe podemos ver directamente lo que hemos hecho co el codigo anternior para ver que en rloc el maximo son 42

In [35]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 38 columns):
act_date               9999 non-null object
source                 9999 non-null object
pos_ctry               9999 non-null object
pos_iata               9999 non-null object
pos_oid                9999 non-null object
rloc                   9999 non-null object
cre_date               9999 non-null object
duration               9999 non-null int64
distance               9999 non-null int64
dep_port               9999 non-null object
dep_city               9999 non-null object
dep_ctry               9999 non-null object
arr_port               9999 non-null object
arr_city               9999 non-null object
arr_ctry               9999 non-null object
lst_port               9999 non-null object
lst_city               9999 non-null object
lst_ctry               9999 non-null object
brd_port               9999 non-null object
brd_city               9999 non-null object
brd_ctry       

In [36]:
b.isnull().sum() # no es normal no tener nulls en una tabla, es una tabla super limpia (primeros 10.000)

act_date               0
source                 0
pos_ctry               0
pos_iata               0
pos_oid                0
rloc                   0
cre_date               0
duration               0
distance               0
dep_port               0
dep_city               0
dep_ctry               0
arr_port               0
arr_city               0
arr_ctry               0
lst_port               0
lst_city               0
lst_ctry               0
brd_port               0
brd_city               0
brd_ctry               0
off_port               0
off_city               0
off_ctry               0
mkt_port               0
mkt_city               0
mkt_ctry               0
intl                   0
route                  0
carrier                0
bkg_class              0
cab_class              0
brd_time               0
off_time               0
pax                    0
year                   0
month                  0
oid                    0
dtype: int64

## 2. Seleccionamos las columnas de interes

In [37]:
b = pd.read_csv('bookings.sample.csv.bz2', sep='^',  usecols = ['arr_port', 'pax', 'year'])
# podemos hacerlo sobre el fichero grande y seleccionar nrows=9999, es lo mismo

In [38]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


- Qué hacemos con los NULL

In [39]:
b = b.dropna()

-PLAN DE ACCION:
        - filtrar 2013
        - groupby sobre arr_port
        - sumar pax
        - ordenar
        - top 10

In [40]:
b = b[b['year']==2013]



In [41]:
len(b)

9999

In [42]:
del b['year'] # year ya no te sirve para nada

In [44]:
top = b.groupby('arr_port').sum().sort_values(by='pax', ascending= False)
top[:10]

,pax
arr_port,
HKG,112
LGA,95
ORD,94
JFK,92
LAX,91
SFO,91
MCO,90
DCA,82
DEN,79


In [45]:
top.reset_index().head()

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,LAX,91


-hay que unir todo lo anterior en una celda para sacar le codigo limpio, desde pd.set.option hasta el top

- Hemos sacado Top 10 con 10.000 datos, ahora hay que extrapolarlo al archivo entero con los chunks
### Chunks

In [46]:
bi = pd.read_csv('bookings.csv.bz2', sep='^',  usecols = ['arr_port', 'pax', 'year'], nrows = 9999, iterator=True )
#el iterator es la clave para que vaya recorriendo los chunks y vaya avanzando por el archivo,no definimos que numero de elementos

In [47]:
type(bi)

pandas.io.parsers.TextFileReader

In [48]:
b = bi.get_chunk(6000) # si hacemos otra vez get empezamos a partir de 6000

In [49]:
type(b) # esto es lo que leiamos antes con el read_csv, un DF

pandas.core.frame.DataFrame

In [50]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


In [51]:
b1 = bi.get_chunk(3000)
b1.head(3)

,arr_port,pax,year
6000,BOS,1,2013
6001,BOS,-1,2013
6002,BOS,-1,2013


In [52]:
b2 = bi.get_chunk(300000000)
b2.shape

(999, 3)

- no devuelve error porque hemos definido con nrows que cuando lleguemos a 9999 para, es decir, no tenemos que iterar con el numero exacto de elementos del fichero, cuando llegue al final del fichero para la iteración

In [53]:
bi = pd.read_csv('bookings.csv.bz2', sep='^',  usecols = ['arr_port', 'pax', 'year'], nrows = 9999, chunksize=3000)
# el iterator=True se puede quitar, funciona igualmente
all_chunks=pd.DataFrame() # aqui van a ir todos los chunks
for i,b in enumerate(bi):
    print(i)
    print(len(b))
    #ahora ponemos el codigo de antes, que vamos a repetir para todos los chunks
    b = b.dropna()
    b = b[b['year']==2013]
    del b['year'] # year ya no te sirve para nada
    top = b.groupby('arr_port').sum().sort_values(by='pax', ascending= False)
    top.reset_index(inplace=True)
    all_chunks = all_chunks.append(top)
    
    
all_result = all_chunks.groupby('arr_port').sum().sort_values(by='pax', ascending= False).reset_index()


0
3000
1
3000
2
3000
3
999


In [54]:
all_chunks.shape

(1344, 2)

In [55]:
all_result.head(10)

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,LAX,91
5,SFO,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


In [ ]:
-Ya lo hemos probado y funciona el codigo, ahora lo hacemos sobre el archivo entero

In [56]:
%%time
bi = pd.read_csv('bookings.csv.bz2', sep='^',  usecols = ['arr_port', 'pax', 'year'],  chunksize=1000000)
# el iterator=True se puede quitar, funciona igualmente
all_chunks=pd.DataFrame() # aqui van a ir todos los chunks
for i,b in enumerate(bi):
    print(i)
    print(len(b))
    #ahora ponemos el codigo de antes, que vamos a repetir para todos los chunks
    b = b.dropna()
    b = b[b['year']==2013]
    del b['year'] # year ya no te sirve para nada
    top = b.groupby('arr_port').sum().sort_values(by='pax', ascending= False)
    top.reset_index(inplace=True)
    all_chunks = all_chunks.append(top)
    
    
all_result = all_chunks.groupby('arr_port').sum().sort_values(by='pax', ascending= False).reset_index()

0
1000000
1
1000000
2
1000000
3
1000000
4
1000000
5
1000000
6
1000000
7
1000000
8
1000000
9
1000000
10
10
CPU times: user 10min 35s, sys: 3.68 s, total: 10min 38s
Wall time: 10min 34s


In [57]:
all_result.head(10)

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0
5,CDG,64490.0
6,BKK,59460.0
7,MIA,58150.0
8,SFO,58000.0
9,DXB,55590.0


## Bonus point del ejercicio 2, GeoBase


In [ ]:
! pip install Neobase

In [58]:
import neobase as nb

In [59]:
from neobase import NeoBase

In [60]:
geoDict = NeoBase()

In [61]:
type(geoDict)

neobase.neobase.NeoBase

In [62]:
geoDict.get('LHR') # nombre aeropuerto

{'__dup__': set(),
 'iata_code': 'LHR',
 'name': 'London Heathrow Airport',
 'lat': '51.4775',
 'lng': '-0.461389',
 'page_rank': 0.44517643489228376,
 'country_code': 'GB',
 'country_name': 'United Kingdom',
 'continent_name': 'Europe',
 'timezone': 'Europe/London',
 'city_code_list': ['LON'],
 'city_name_list': ['London'],
 'location_type': ['A'],
 'currency': 'GBP'}

In [63]:
type(geoDict.get('LHR') )

dict

In [64]:
geoDict.get('LHR')['name']

'London Heathrow Airport'

In [65]:
all_result.head()

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0


In [66]:
all_result['arr_port'][0]

'LHR     '

In [ ]:
- Esto hay que arreglarlo, columna con espacios

In [67]:
all_result['arr_port'][0].strip() #.strip quita los espacios en blanco a ambos lados

'LHR'

- Si tenemos espacios en blanco en el medio, funciones split y join

In [68]:
'skdfsljf d    h    dkfdkfjdfjdfk'.split()

['skdfsljf', 'd', 'h', 'dkfdkfjdfjdfk']

In [69]:
' '.join('skdfsljf d    h    dkfdkfjdfjdfk'.split())

'skdfsljf d h dkfdkfjdfjdfk'

In [70]:
all_result['arr_port'] =all_result['arr_port'].str.strip()

In [71]:
all_result['AirportName'] = all_result['arr_port'].map(lambda x:geoDict.get(x)['name']) 
# este error quiere decir que el eropuerto CPQ no esta en el otro sitio

KeyError: 'Key not found: CPQ'

In [72]:
all_result[all_result['arr_port']=='CPQ'] # da error porque CPQ no tiene name, comprobar!!!!!!

,arr_port,pax
2257,CPQ,-20.0


In [73]:
all_result = all_result[all_result['arr_port']!='CPQ']

-Ahora sí aplicamos el mismo codigo

In [74]:
all_result['AirportName'] = all_result['arr_port'].map(lambda x:geoDict.get(x)['name']) 


In [75]:
all_result.head()

,arr_port,pax,AirportName
0,LHR,88809.0,London Heathrow Airport
1,MCO,70930.0,Orlando International Airport
2,LAX,70530.0,Los Angeles International Airport
3,LAS,69630.0,McCarran International Airport
4,JFK,66270.0,John F. Kennedy International Airport


In [76]:
all_result.shape

(2273, 3)

In [77]:
all_result.to_csv('top_airports.csv', sep='^',index=False)

In [ ]:
! head -2 top_airports.csv

### - Ejercicio 5: Write a Web Service: Wrap the output of the second exercise in a web service that returns the data in JSON format (instead of printing to the standard output). The web service should accept a parameter n>0. For the top 10 airports, n is 10. For the X top airports, n is X 

In [ ]:
#el webservice es un api, lo que esta trabajando en la shell cuando abrimos notebook es un WS, el local host de arriba

In [79]:
from flask import Flask

In [80]:
app =Flask('My first web service')
@app.route('/hello', methods=['GET'])
#GET The browser tells the server to just get the infomation stored
def get_hello():
             return 'Hello DS from the service!'


In [ ]:
app.run()
#running os http://127.0.0.1:5000/

 * Serving Flask app "My first web service" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Feb/2020 19:50:19] "GET /hello HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2020 19:51:28] "GET /ret_number/10 HTTP/1.1" 404 -


In [ ]:
app =Flask('My first web service')
@app.route('/hello', methods=['GET'])
#GET The browser tells the server to just get the infomation stored
def get_hello():
             return 'Hello DS from the service!'
@app.route('/ret_number/<int:n>',methods=['GET'])
def get_number(n):
    return 'i got %d'%n

In [ ]:
app.run()

In [ ]:
# el Viernes vamos a aplicar esto al archivo del top ten , intentar